In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import metrics
import itertools
from sklearn.linear_model import PassiveAggressiveClassifier
import re
import time
import pickle
import zlib

import nltk 
from nltk.tokenize import word_tokenize 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import seaborn as sb
%matplotlib inline

In [2]:
df = pd.read_csv('Data_Preperation/final_data')
df.shape
df

,Unnamed: 0,text,labels
0,1230,HORRIFYING! Raining Spiders In Brazil HORRIFYI...,real
1,5080,Report: Freddie Gray sustained injury in polic...,real
2,9200,"Clinton, FBIGate and the true depth of the Oba...",fake
3,4745,Paris climate summit: What to expect from hist...,real
4,7664,Why Isn’t NSA Surveillance an Election Issue? ...,fake
...,...,...,...
10339,3691,Neck Deep scrap Nottingham show over 'heavy-ha...,fake
10340,2605,White House Tax Reform Goals focus on Rates an...,real
10341,9052,Initial prognosis poor for Obama’s immigration...,real
10342,9954,Break the Silence or Support Self-Determinatio...,fake


In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,text,labels
0,HORRIFYING! Raining Spiders In Brazil HORRIFYI...,real
1,Report: Freddie Gray sustained injury in polic...,real
2,"Clinton, FBIGate and the true depth of the Oba...",fake
3,Paris climate summit: What to expect from hist...,real
4,Why Isn’t NSA Surveillance an Election Issue? ...,fake
...,...,...
10339,Neck Deep scrap Nottingham show over 'heavy-ha...,fake
10340,White House Tax Reform Goals focus on Rates an...,real
10341,Initial prognosis poor for Obama’s immigration...,real
10342,Break the Silence or Support Self-Determinatio...,fake


In [4]:
# Finding NaN values in the dataset

nan = df['text'].loc[df['text'].isnull()]
print(" There are {} NaN values in text".format(len(nan)))

nan = df['labels'].loc[df['labels'].isnull()]
print(" There are {} NaN values in labels".format(len(nan)))

 There are 21 NaN values in text
 There are 0 NaN values in labels


In [5]:
df = df.dropna()

In [6]:
# Finding NaN values in the dataset

nan = df['text'].loc[df['text'].isnull()]
print(" There are {} NaN values in text".format(len(nan)))

nan = df['labels'].loc[df['labels'].isnull()]
print(" There are {} NaN values in labels".format(len(nan)))

 There are 0 NaN values in text
 There are 0 NaN values in labels


In [7]:
df.shape

(10323, 2)

In [8]:
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['text'].values):
    #sent = decontracted(sentance)
    sentance = sentance.replace('\\r', ' ')
    sentance = sentance.replace('\\"', ' ')
    sentance = sentance.replace('\\n', ' ')
    sentance = re.sub('[^A-Za-z0-9]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    #sentance = ' '.join(e for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.lower().strip())

df['text'] = preprocessed_text

100%|██████████████████████████████████████████████████████████████████████████| 10323/10323 [00:03<00:00, 2953.87it/s]
D:\python\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [9]:
y = df['labels']
X = df['text']

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,)

In [11]:
tfidf_vect = TfidfVectorizer(max_features=12000)
tfidf_train = tfidf_vect.fit_transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names())

In [12]:
filepath = 'web\\vectorizer.pkl'
pickle.dump(tfidf_vect, open(filepath, 'wb'))

In [13]:
clf = MultinomialNB() 
clf.fit(tfidf_train, y_train)                      
pred = clf.predict(tfidf_test)                     
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.8579270261543429
[[1311  198]
 [ 242 1346]]


In [14]:
clf = GradientBoostingClassifier()
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.8850500484339684
[[1331  178]
 [ 178 1410]]


In [15]:
clf1  = RandomForestClassifier(n_estimators=1725, random_state=1, n_jobs=-1, class_weight='balanced')
clf1.fit(tfidf_train, y_train)
clf1.score(tfidf_test, y_test)

0.9118501775912173

In [16]:
clf = SVC(C=100, gamma=0.1,probability=True)
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.9228285437520181
[[1391  118]
 [ 121 1467]]


In [17]:
filepath = 'web\\classifier.pkl'
pickle.dump(clf1, open(filepath, 'wb'))